In [1]:
import pandas as pd
from storage.storage import Storage

In [2]:
nodes_df = pd.read_csv('substations.csv')
nodes_df['node_id'] = nodes_df.reset_index().index
edges_df = pd.read_csv('transmission.csv')
edges_df['edge_id'] = edges_df.reset_index().index
loc_df = pd.read_csv('lat-long.csv')

In [3]:
edges_df

,Line Name,Voltage (kV),Substation Name,Facility Code,Planning Area,TFO,Capability (MW),edge_id
0,1002L,240,Amoco Empress,163S,48-Empress,AltaLink,158,0
1,1002L,240,Jenner,275S,48-Empress,AltaLink,265,1
2,1005AL,240,Little Bow,991S,49-Stavely,AltaLink,84,2
3,1005L,240,Picture Butte,120S,54-Lethbridge,AltaLink,311,3
4,1005L,240,Milo,356S,47-Brooks,AltaLink,117,4
...,...,...,...,...,...,...,...,...
474,9L99,240,Sheerness,807S,43-Sheerness,ATCO,83,474
475,CHD-1,69,Magrath,225S,55-Glenwood,AltaLink,15,475
476,CHD-1,69,Taylor Hydro Plant,NaN,55-Glenwood,AltaLink,15,476
477,PCES01L,138,Namaka,428S,45-Strathmore/Blackie,AltaLink,5,477


In [4]:
db = Storage()
db.db = 'grid.db'

In [5]:
db.save_data('nodes', nodes_df, replace=True)
db.save_data('edges', edges_df, replace=True)
db.save_data('loc', loc_df, replace=True)

0

In [10]:
q = '''
SELECT * from nodes
left join loc on (nodes.`Facility Code` = loc.`Substation`)
'''
merged_df = db.run_query(q)

In [11]:
merged_df.head(5)

,Name,Facility Code,TFO,Planning Area,Bus Number,Voltage (kV),Capability (MW),node_id,Substation,Shape,Latitude (generated),Longitude (generated)
0,Al Rothbauer,321S,AltaLink,04-Medicine Hat,976,138,220,0,321S,Point,50.062035,-110.721055
1,Amoco Empress,163S,AltaLink,48-Empress,262,240,160,1,163S,Point,50.678140,-110.026638
2,Amoco Empress,163S,AltaLink,48-Empress,267,138,125,2,163S,Point,50.678140,-110.026638
3,Anderson,801S,ATCO,43-Sheerness,1484,240,1006,3,801S,Point,51.429079,-111.799296
4,Anderson,801S,ATCO,43-Sheerness,1485,144,85,4,801S,Point,51.429079,-111.799296


In Location list but not in either list

In [12]:
db.save_data('nodes_loc', merged_df, replace=True)

0

In [13]:
merged_df.nunique()

Name                     192
Facility Code            191
TFO                        4
Planning Area             18
Bus Number               228
Voltage (kV)               5
Capability (MW)          149
node_id                  228
Substation               190
Shape                      1
Latitude (generated)     190
Longitude (generated)    190
dtype: int64

In [26]:
q = '''
SELECT edges.`Line Name`, edges.`Voltage (kV)` as `edge_voltage_kv`, edges.`Capability (MW)` as `edge_cap_mw`, edges.edge_id, nodes_loc.* from edges
inner join nodes_loc on (nodes_loc.`Facility Code` = edges.`Facility Code`)
'''

full_df = db.run_query(q)

In [36]:
full_df.head(10)

,Line Name,edge_voltage_kv,edge_cap_mw,edge_id,Name,Facility Code,TFO,Planning Area,Bus Number,Voltage (kV),Capability (MW),node_id,Substation,Latitude (generated),Longitude (generated)
0,1002L,240,158,0,Amoco Empress,163S,AltaLink,48-Empress,262,240,160,1,163S,50.678140,-110.026638
1,1002L,240,158,0,Amoco Empress,163S,AltaLink,48-Empress,267,138,125,2,163S,50.678140,-110.026638
2,1002L,240,265,1,Jenner,275S,AltaLink,48-Empress,260,240,271,107,275S,50.710576,-111.091598
3,1002L,240,265,1,Jenner,275S,AltaLink,48-Empress,292,138,104,108,275S,50.710576,-111.091598
4,1005AL,240,84,2,Little Bow,991S,AltaLink,49-Stavely,549026,240,84,116,991S,50.264729,-112.764489
5,1005L,240,311,3,Picture Butte,120S,AltaLink,54-Lethbridge,451,240,311,155,120S,49.827353,-112.734982
6,1005L,240,117,4,Milo,356S,AltaLink,47-Brooks,943,240,1327,131,356S,50.521809,-112.767802
7,100L,138,58,5,Tilley,498S,AltaLink,47-Brooks,275,138,127,195,498S,50.490566,-111.652846
8,100L,138,62,6,Suffield,895S,AltaLink,04-Medicine Hat,270,138,63,187,895S,50.231783,-111.172141
9,1011L,240,158,7,Amoco Empress,163S,AltaLink,48-Empress,262,240,160,1,163S,50.678140,-110.026638


In [32]:
full_df.drop(['Shape'], axis=1, inplace=True)

In [33]:
db.save_data('merged', full_df, replace=True)

0